In [1]:
#r "nuget: Microsoft.ML"
#r "nuget: LemmaSharpPreBuilt-std"
#r "NeuterBot.StandardSearch.dll"
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms.Text;
using NeuterBot.Search;
var mlContext = new MLContext();



The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package LemmaSharpPreBuilt-std version 1.0.1

Installed package Microsoft.ML version 1.5.2

# Step 1: Create Knowledge Base

Each answer has one more questions associated with it.


In [2]:
class QnASet {
    public IEnumerable<string> Questions {get;set;}
    public string Answer {get;set;}
 }

var qnaSet1 = new QnASet() {Questions = new [] {"What eats rats?", "What preys on rats?"},Answer = "Cats."};
var qnaSet2 = new QnASet() {Questions = new [] {"What do rats eat?"}, Answer = "Everything."};
display(qnaSet1);
display(qnaSet2);

Questions,Answer
"[ What eats rats?, What preys on rats? ]",Cats.


Questions,Answer
[ What do rats eat? ],Everything.


We don't do any processing on the text of the answers - the users see that verbatim.
We just need to track which answer corresponds to each question.
So we'll work from now on with list of pairs of single questions and answers.

In [3]:
class QnaPair  {
     public string Question {get;set;}
    public string Answer {get;set;}
    public string NormalisedQuestion {get;set;}
    public string[] Tokens {get;set;}
    public string[] TokensWithStopWordsRemoved {get;set;}
    public string[] LemmatizedTokens {get;set;}
    
    public float[] TFVector {get;set;}
    public float[] IDFVector {get;set;}
    public float[] TFIDFVector {get;set;}
}
var pairs = (new [] {qnaSet1, qnaSet2}).SelectMany(qnaset => qnaset.Questions.Select(q => new QnaPair() {Question = q, Answer = qnaset.Answer}));




 1. Make a vocab of all ngrams in the knowledge base.
 1. Nomalise the questions (remove punctuation and convert to lower case).
 1. Split to tokens.
 1. Remove stop words.
 1. Lemmatize.
 1. Find TF vectors.
 1. Find IDF vectors.
 1. Multiply both vectors to get a TIIDF vector for each question.
    
    ![TFIDF Formulas](idfFormula.png)




In [4]:
// No build-in lemmatizer in ML.Net, so we have to add our own.
Action<QnaPair, QnaPair> customLemmatizerFunction = (x, y) =>
{
    var lemmatizer = new WrappedLemmatizer();
    if (x.TokensWithStopWordsRemoved != null)
    {
        y.Question = x.Question;
        y.Answer = x.Answer;
        y.NormalisedQuestion = x.NormalisedQuestion;
        y.Tokens = x.Tokens;
        y.TokensWithStopWordsRemoved = x.TokensWithStopWordsRemoved;      
        y.LemmatizedTokens = x.TokensWithStopWordsRemoved.Select(t => lemmatizer.Lemmatize(t)).ToArray();
    }
    else
    {
        y.LemmatizedTokens = null;
    }
};

var textPipeline = mlContext.Transforms.Text.NormalizeText("NormalisedQuestion", "Question", TextNormalizingEstimator.CaseMode.Lower, false, false, true)
    .Append(mlContext.Transforms.Text.TokenizeIntoWords("Tokens", "NormalisedQuestion"))
    .Append(mlContext.Transforms.Text.RemoveDefaultStopWords("TokensWithStopWordsRemoved","Tokens"))
    .Append(mlContext.Transforms.CustomMapping<QnaPair, QnaPair>(customLemmatizerFunction, "CMContract"))
    .Append(mlContext.Transforms.Text.ProduceWordBags(outputColumnName:  "TFVector", inputColumnName: "LemmatizedTokens", ngramLength: 2, skipLength: 0, useAllLengths: true, maximumNgramsCount: 100000, weighting: NgramExtractingEstimator.WeightingCriteria.Tf ))
    .Append(mlContext.Transforms.Text.ProduceWordBags(outputColumnName:  "IDFVector", inputColumnName: "LemmatizedTokens", ngramLength: 2, skipLength: 0, useAllLengths: true, maximumNgramsCount: 100000, weighting: NgramExtractingEstimator.WeightingCriteria.Idf ))
    .Append(mlContext.Transforms.Conversion.MapValueToKey("LemmatizedTokensKeyed","LemmatizedTokens"))
    
    .Append(mlContext.Transforms.Text.ProduceNgrams("TFIDFVector",
        "LemmatizedTokensKeyed",
        ngramLength: 2,
        useAllLengths: true,
        weighting: NgramExtractingEstimator.WeightingCriteria.TfIdf));

var dataView = mlContext.Data.LoadFromEnumerable(pairs);
var textTransformer = textPipeline.Fit(dataView); 
var transformedDataView = textTransformer.Transform(dataView);

// show the vocab
var featuresColumn = transformedDataView.Schema["TFIDFVector"];
VBuffer<ReadOnlyMemory<char>> slotNames = default;
featuresColumn.GetSlotNames(ref slotNames);
var slotNamesMap = slotNames.Items();
List<string> vocab = new List<string>();
foreach (var ngram in slotNamesMap)
{
    vocab.Add(ngram.Value.ToString());
}

Console.WriteLine("Vocab:");
display(vocab);

Console.WriteLine("Knowledge Base:");
var enumerated = mlContext.Data.CreateEnumerable<QnaPair>(transformedDataView,false).ToList();
display(enumerated);



Vocab:


index,value
0,eat
1,eat|rat
2,rat
3,prey
4,prey|rat
5,rat|eat


Knowledge Base:


index,Question,Answer,NormalisedQuestion,Tokens,TokensWithStopWordsRemoved,LemmatizedTokens,TFVector,IDFVector,TFIDFVector
0,What eats rats?,Cats.,what eats rats,"[ what, eats, rats ]","[ eats, rats ]","[ eat, rat ]","[ 1, 1, 1, 0, 0, 0 ]","[ 0.4054651, 1.0986123, 0, 0, 0, 0 ]","[ 0.4054651, 1.0986123, 0, 0, 0, 0 ]"
1,What preys on rats?,Cats.,what preys on rats,"[ what, preys, on, rats ]","[ preys, rats ]","[ prey, rat ]","[ 0, 0, 1, 1, 1, 0 ]","[ 0, 0, 0, 1.0986123, 1.0986123, 0 ]","[ 0, 0, 0, 1.0986123, 1.0986123, 0 ]"
2,What do rats eat?,Everything.,what do rats eat,"[ what, do, rats, eat ]","[ rats, eat ]","[ rat, eat ]","[ 1, 0, 1, 0, 0, 1 ]","[ 0.4054651, 0, 0, 0, 0, 1.0986123 ]","[ 0.4054651, 0, 0, 0, 0, 1.0986123 ]"



(20,20): warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'Microsoft.DotNet.Interactive.Formatting' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=5.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



# Step 2: Query

Firstly we get the TFIDF Vector for the query, using the vocab in the knowledge base.

In [5]:
class Query {
    public string QueryText {get;set;}
    public float[] TFIDFVector {get;set;}
}

var testQueries = new [] {
    "What do rats eat?",
    "What do I eat?",
    "Can I have a cake?"
}.Select(i => new Query() {QueryText = i});

var predictionEngine =  mlContext.Model.CreatePredictionEngine<QnaPair,Query>(textTransformer);

List<Query> predictionResults = new List<Query>();
foreach (var q in testQueries)
{
    Query prediction = new Query() {QueryText = q.QueryText};
    predictionEngine.Predict(new QnaPair() {Question = q.QueryText}, ref prediction);
    predictionResults.Add(prediction);

}

display(predictionResults);

index,QueryText,TFIDFVector
0,What do rats eat?,"[ 0.4054651, 0, 0, 0, 0, 1.0986123 ]"
1,What do I eat?,"[ 0.4054651, 0, 0, 0, 0, 0 ]"
2,Can I have a cake?,"[ 0, 0, 0, 0, 0, 0 ]"


Then we compare the TFIDF Vector for the query to the TFIDF vectors for each question in the knowledge base.

![Cosign Similarity formula](cosignSimilarityFormula.png)

![Cosign Similarity diagram](cosignDiagram.png)

[Source](https://towardsdatascience.com/understanding-cosine-similarity-and-its-application-fd42f585296a)



In [6]:
double CalculateSimilarity(float[] queryVector, float[] documentVector)
{
    Func<float[], float> calulateDotProduct = (qv) =>
    {
        float dotProduct = 0;
        for (int i = 0; i < queryVector.Count(); i++)
        {
            dotProduct += (qv[i] * documentVector[i]);
        }
        return dotProduct;
    };

    Func<float[], double> calculateMagnitude = v => {
        return Math.Sqrt(v.Sum(f => f * f));
    };
    
    var css = calulateDotProduct(queryVector) / ((calculateMagnitude(queryVector) * calculateMagnitude(documentVector)));
    return (Double.IsNaN(css) ? 0 : css);
}


var comp = predictionResults.Select(query => new { 
    Query = query.QueryText,  
    ComparisonScores = enumerated.Select(r => new {Question = r.Question, Similarity = CalculateSimilarity(r.TFIDFVector, query.TFIDFVector)})
    });

//var comparisions = predictionResults.SelectMany(i => enumerated.Select(d => new {QueryText = i.QueryText, QuestionInKB = d.Question, Similarity = CalculateSimilarity(i.TFIDFVector,d.TFIDFVector)}));

display(comp);


index,Query,ComparisonScores
0,What do rats eat?,"[ { Question = What eats rats?, Similarity = 0.11988320132103283 }, { Question = What preys on rats?, Similarity = 0 }, { Question = What do rats eat?, Similarity = 0.9999999999999999 } ]"
1,What do I eat?,"[ { Question = What eats rats?, Similarity = 0.3462415361002097 }, { Question = What preys on rats?, Similarity = 0 }, { Question = What do rats eat?, Similarity = 0.3462415361002097 } ]"
2,Can I have a cake?,"[ { Question = What eats rats?, Similarity = 0 }, { Question = What preys on rats?, Similarity = 0 }, { Question = What do rats eat?, Similarity = 0 } ]"
